<div align=center>
<img src=https://github.com/datoujinggzj/WhaleDataScienceProject/blob/master/pic/click.gif?raw=true width='100' />
</div>

---

<div align=center>
<a href='https://zg104.github.io'> 
<img src=https://github.com/datoujinggzj/WhaleDataScienceProject/blob/master/pic/17.png?raw=true/>
</div>

<div class="alert alert-block alert-warning">
<center><b>【第四阶段】项目日志</b></center>
    
    
第四阶段：【数据分析全解析】
   
第三节阶段核心目的：全方位解析你所不知道的数据分析全链路，第三阶段数据分析部分结束。
    
难度（最高5星）：⭐⭐⭐
    
第三阶段周期：2022年4月22日 至 2022年4月29日（北京时间）


Good Luck!
</div>

接着上一阶段，我们了解了如何去处理类别不平衡数据，这一阶段，我们把重点放在数据分析的层面，那么第一步就是处理缺失值！


# 缺失值处理

我们知道在这个数据集中，-1代表着缺失值，那么我们对付缺失值无非就是两招：

- 移除
- 填充

移除比较简单，要么直接干掉整个列（使用`drop(axis=1)`），要么去除所有含缺失值的行（使用`dropna()`）。

前提是，缺失值在整个列的比例达到40%以上，我们才可以drop列，不然我们就要采取下一种方法，也就是填充。

**建议大家先阅读一下下面的文章链接**👇👇👇

https://gitee.com/gzjzg/data-preparation-crash-course/tree/master/HandleMissingValues

In [1]:
# import packages

# data processing
import pandas as pd
import numpy as np
from datetime import timedelta, datetime


import re

# data visualization
import plotly.graph_objs as go
from plotly.graph_objs import Bar, Layout
from plotly import offline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

# change text color
import colorama
from colorama import Fore, Style

# IPython
from IPython.display import IFrame

In [2]:
# 读取数据

train = pd.read_csv(r"C:\Users\gzjgz\OneDrive\Desktop\WhaleDataScienceProject-master\Python_data\porto-seguro-safe-driver-prediction_data\train.gz")
test = pd.read_csv(r"C:\Users\gzjgz\OneDrive\Desktop\WhaleDataScienceProject-master\Python_data\porto-seguro-safe-driver-prediction_data\test.gz")

In [3]:
# 全集
fullset = pd.concat([train,test],ignore_index=True)

## 缺失值诊断

那么在诊断数据缺失值时，一定要搞清楚为什么会存在缺失值，这将极大影响我们接下来如何处理缺失值！

先介绍一下「缺失类型」

- MCAR (Missing completely at Random)
- MAR (Missing at Random)
- MNAR (Missing not at Random)

MCAR 的意思是“缺失与任何值之间没有关系”。因此，在这种情况下，可以删除缺失值。为此，可以删除列缺失值；最小化丢失数据或删除行缺失值。

MAR 的意思是“缺失与其他观察到的数据之间存在系统关系，但与缺失数据无关”。

最后，MNAR 的意思是“缺失与它的值之间存在关系，缺失或非缺失”。

### 使用元数据

这里可以借助meta元数据来查看缺失值情况！

In [4]:
from data_management import meta

In [5]:
metadata = meta(train,test)

<div class="alert alert-block alert-success"><b>Step 1</b>: 尝试基于meta找出所有含有缺失值的变量，并从大到小排序。    

</div>

In [6]:
missing_data = "你的代码！"

In [7]:
missing_data

'你的代码！'

<div class="alert alert-block alert-success"><b>Step 2</b>: 所以哪些变量需要drop掉？哪些需要填充？该采取什么样的方法填充？在meta中能找到一些启发么？

</div>

In [8]:
cols_to_drop = "你的代码！"

In [9]:
cols_to_imp = missing_data.index[2:].to_list()
cols_to_imp_3m = missing_data[missing_data.imputation == ('mode', 'median', 'mean')].index.to_list()
cols_to_imp_2m = missing_data[missing_data.imputation == ('mode', 'median')].index.to_list()
cols_to_imp_1m = missing_data[missing_data.imputation == ('mode')].index.to_list()

TypeError: 'builtin_function_or_method' object is not subscriptable

以上，是单变量填充的思考方式，也是比较简单易理解的方法。

什么是**单变量填充（univariate imputation）**？

**就是当前列的缺失值只能通过当前列的信息填充，跟其他列无关。**

后面，会给大家介绍如何单变量填充！

### 使用missingno

参考：https://github.com/datoujinggzj/DataScienceCrashCourse/blob/master/HandleMissingValues/03-missingno.ipynb

In [ ]:
import missingno as msno

In [ ]:
msno.bar(fullset.drop(['id','target'],axis=1).replace(-1,np.nan).sample(1000),figsize = [15,20]) 

In [ ]:
msno.matrix(fullset.drop(['id','target'],axis=1).sample(200).replace(-1,np.nan)) 

# 更清晰的展示缺失值的分布，但是在这里是无效的，主要用于时间序列数据，但这个数据不是！

In [ ]:
msno.heatmap(fullset[cols_to_imp].replace(-1,np.nan))

In [ ]:
msno.dendrogram(fullset[cols_to_imp].replace(-1,np.nan))

## 缺失值填充

- 单变量
- 多变量

> 单变量填充：每一列填充缺失值都必须使用本列的信息，不能使用其他列的信息。


> 多变量填充：可以借助其他列的信息。

### 多变量填充

通过上图可以看出每一对变量之间的缺失值关联程度，比如：

ps_car_01_cat的缺失就会导致ps_ind_04_cat的缺失，是正向的关系，且关联程度很大！

所以，接下来给大家介绍【多变量填充】，如果变量之间的缺失值生成原因有关联（上方的heatmap）

从上图可以看出，我们尝试用多变量填充方法去填充有关联的变量！

- ps_ind_05_cat | ps_car_07_cat
- ps_car_01_cat | ps_ind_02_cat | ps_ind_04_cat
- ps_reg_03 | ps_car_14 （连续型）

以上这两组，组间有较强的缺失值关联度，最后一组是连续型变量，我们现在来介绍方法：

这里为大家介绍以下几个多变量填充方法：

https://scikit-learn.org/stable/modules/impute.html

---

- Multivariate Imputation By Chained Equations (MICE) 
- KNN

那么我们分别来用这两个包实现一下！

In [ ]:
set1 = ['ps_ind_05_cat','ps_car_07_cat']
set2 = ['ps_car_01_cat','ps_ind_02_cat','ps_ind_04_cat']
set3 = ['ps_reg_03','ps_car_14']

#### MICE

MICE是非常简单好理解且比较常用的多变量填充方法，在sklearn和impyute上都可以实现，让我们来看看其原理吧！

In [ ]:
IFrame(width="853",height="480",src = "https://www.youtube.com/embed/WPiYOS3qK70")

In [ ]:
%%time

from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, random_state=123)

imp_mean = IterativeImputer(estimator=rf, missing_values=-1, random_state=0)
set1_imp = imp_mean.fit_transform(train[set1])

In [ ]:
%%time
set2_imp = imp_mean.fit_transform(train[set2])

仅仅从2个变量加到3个变量，运行时间就指数型增加，expensive！

可以看到，对于set1来说，这方法并不适用于填充类别型变量，直接把整数给填充成小数了，如果再把小数直接round，那么就跟直接众数填充（单变量填充）没啥区别。

对于set2，貌似没有直接按照众数填充，把缺失值填充到8左右，但是后面的缺失值实在太少了，几十万的数据，只有几百个缺失值，确实不够看，而且这种填充对本来的数据影响不大。

那么再来试试2个连续型变量吧！

In [ ]:
%%time
set3_imp = imp_mean.fit_transform(train[set3])

说明这个算法没有收敛，那么我们来看一下分布！

In [ ]:
pd.DataFrame(set3_imp,columns = set3)

In [ ]:
plt.subplot(3,1,1)
sns.kdeplot(train[set3[0]])
plt.subplot(3,1,2)
sns.kdeplot(train[set3[0]].replace(-1,np.nan).fillna(train[set3[0]].mean()))
plt.subplot(3,1,3)
sns.kdeplot(pd.DataFrame(set3_imp,columns = set3)[set3[0]])

plt.tight_layout()

In [ ]:
plt.subplot(3,1,1)
sns.kdeplot(train[set3[1]])
plt.subplot(3,1,2)
sns.kdeplot(train[set3[1]].replace(-1,np.nan).fillna(train[set3[1]].mean()))
plt.subplot(3,1,3)
sns.kdeplot(pd.DataFrame(set3_imp,columns = set3)[set3[1]])

plt.tight_layout()

可以看出，MICE可以更好地吻合原变量的概率分布，所以我们选择使用MICE填充！

In [ ]:
train[set3] = pd.DataFrame(set3_imp,columns = set3)

#### KNN

In [ ]:
%%time
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(missing_values=-1,n_neighbors=2000)
# imputer.fit_transform(train[set1])

# Wall time: 7min 35s

这个计算时间真的是让我望而却步了，大家在面对大型数据集还是慎用吧，KNN真是让我害怕！

### 单变量填充

单变量填充就比较简单了，请大家直接看我的笔记！

https://gitee.com/gzjzg/data-preparation-crash-course/blob/master/HandleMissingValues/01-Missing_Data_Proceesing_Basis.ipynb

主要有：

- fillna
- replace
- SimpleImputer
- Intepolate

这里我们主要使用SimpleImputer来填充！

不同的strategy形成不同的填充模式，

- If "<font color=red>mean</font>", then replace missing values using the mean along each column. Can only be used with numeric data.
- If "<font color=red>median</font>", then replace missing values using the median along
  each column. Can only be used with numeric data.
- If "<font color=red>most_frequent</font>", then replace missing using the most frequent
  value along each column. Can be used with strings or numeric data.
  If there is more than one such value, only the smallest is returned.
- If "<font color=red>constant</font>", then replace missing values with fill_value. Can be
  used with strings or numeric data.

In [ ]:
# 缺失值处理（1）
IFrame(width="853",height="480",src = "http://v.xiaohongshu.com/01e23d740163edea018370037fc00fcb53_259.mp4?sign=ddc7525337222487fe99a77c32fa499e&t=6266c580")

In [ ]:
# 缺失值处理（2）
IFrame(width="853",height="480",src = "http://v.xiaohongshu.com/01e24207c120206a018370037fd1f059f3_259.mp4?sign=b402a6d59e6449d0759e2a5953ae6a1c&t=6266c580")

In [ ]:
# 缺失值处理（3）
IFrame(width="853",height="480",src = "http://v.xiaohongshu.com/pre_post/01e248713d22925d018370037feafe1133_259.mp4?sign=ceb965f12f645a52e0a76af7ac953a53&t=6266c580")

In [ ]:
from sklearn.impute import SimpleImputer 

In [ ]:
mode_imputer = SimpleImputer(missing_values = -1, strategy='most_frequent',add_indicator=True)

In [ ]:
missing_data[4:]
# 这些是还没填充的变量，我们就都用mode来填充，
# 因为最后两个缺失值太少了，没有影响。

In [ ]:
imp = mode_imputer.fit_transform(train[missing_data[4:].index])

In [ ]:
train[missing_data[4:].index]=pd.DataFrame(imp).iloc[:,:len(missing_data[4:].index)]

In [ ]:
train[missing_data[4:].index] = train[missing_data[4:].index].astype('int64')

In [ ]:
# drop columns
train.drop(cols_to_drop,axis=1,inplace=True)

In [ ]:
# check out if we still have -1 
(train == -1).sum().sum()

In [ ]:
train.to_csv('train_imp.csv')

<div class="alert alert-block alert-info"> 
<p style="font-size:20px; display:inline">💡</p> 思考：

- add_indicator的作用
- strategy不同造成什么影响
</div>

<div class="alert alert-block alert-info"> 
<p style="font-size:20px; display:inline">💡</p> 思考：

第33行代码是什么意思呢？所以在用SimpleImputer时候要注意什么？
</div>

<div class="alert alert-block alert-info"> 
<p style="font-size:20px; display:inline">💡</p> 思考：

fit_transform、fit有什么区别？
</div>

<div class="alert alert-block alert-success"><b>作业：</b>: 
    
尝试把测试集 test 也填充一下吧！
    
</div>

此时，我们填充缺失值后的train不存在缺失值了，下一阶段我们来讲【异常值处理】。

<div align=center>
<img src=https://github.com/datoujinggzj/WhaleDataScienceProject/blob/master/pic/ds_repo_qr.png?raw=true width='800' />
</div>